In [1]:
import multiprocessing
import scipy.optimize as opt
multiprocessing.set_start_method('fork')


In [2]:
from basic_functionalities import *
import bjmm_mo
import mo
import bm_depth_4
import bm_depth_2
import bm_plus_depth_4

In [3]:
def optimize(time,num_params,constraints):
    start = r(0.001,0.01,num_params)
    bounds = [(0, 1)]*num_params
    
    result = opt.minimize(time, start, 
            bounds= bounds, tol=1e-10, 
            constraints=constraints, options={'maxiter':1000})
    return result

def set_env(module, rate):
    module.k = lambda x : rate
    module.w = lambda x : Hi(1-rate)

def get_data(module):
    time        = module.time
    constraints = module.constraints
    num_vars    = module.num_vars
    return time, constraints, num_vars

def parallel():
    time, constraints, num_vars= get_data(module)
    return optimize(time,num_vars,constraints)

#finds minimum running time for chosen module and given rate k
def find_minimum_for_rate(k):
    set_env(module,k)
    
    count=0
    minval=1
    data=None

    while count<maxiter:
        candidates=[]
        with multiprocessing.Pool(PROCESSES) as pool:
            params = range(PROCESSES)
            results = [pool.apply_async(parallel) for p in range(PROCESSES*elements_per_process)]
            for i in results:
                candidates.append(i.get())
        for i in candidates:
            if i is None:
                continue
            if time(i.x)<minval and i.success:
                minval=time(i.x)
                data=i.copy()
            count+=1
    return data

### Customize here

In [4]:
# Set module and threads/proccesses used for running the optimization
# Each thread runs the optimization ``elements_per_process`` many times 
# Note that the optimization has to be run multiple times to ensure the 
# true global minimum is found, hence ``PROCESSES*elements_per_process`` 
# should be large enough

module = mo
PROCESSES = 8
elements_per_process=1

time,constraints, num_vars= get_data(module)
memory=module.memory
maxiter=PROCESSES*elements_per_process


In [5]:
#Example: finds optimal parameters / running time for rate 0.42
find_minimum_for_rate(0.42)

{'x': array([0.06454529, 0.00406344, 0.02024555, 0.08720324, 0.10216953]),
 'fun': 0.09666529696440852,
 'jac': array([1.56989038, 9.23391362, 5.72679329, 0.08520855, 0.08520855]),
 'nit': 74,
 'nfev': 660,
 'njev': 73,
 'status': 0,
 'message': 'Optimization terminated successfully',
 'success': True}

In [ ]:
#create list with optimum for all rates
k=0.01
Lfd=[]
while k<1:
    data=find_minimum_for_rate(k)
    Lfd.append([k,data])
    k+=0.01

In [ ]:
#extract time and memory lists from Lfd
Ltime=[]
Lmem=[]
for i in Lfd:
    if i[1] is not None:
        set_env(module,i[0])
        x=module.set_vars(*(i[1]['x']))
        Ltime.append([i[0],time(x)])
        Lmem.append([i[0],memory(x)])

### Optimal parameters for worst case complexity of Both-May and Both-May+

In [6]:
#Worst case running time Both-May
module=bm_depth_4
set_env(module,0.42)
x=[0.05179949616832484, 0.0009721235534018956, 0.01011389586902574,
          0.021291015439529482, 0.04786139166993479, 0.0013155102710478903,
          0.002672149147757666, 0.08868019551275919, 0.004811791704030416,
          0.006014685075786457, 0.05279675394684703, 0.10177737427236798,
          0.12367201215526315, 0.006510417020142931, 0.012199921088212254,
          0.015043975055759032]
x=module.set_vars(*x)
module.time(x),module.memory(x)

(0.09502623352766121, 0.0752976719520608)

In [7]:
#Worst case running time Both-May+
module=bm_plus_depth_4
set_env(module,0.43)
x=[4.51370747e-02, 1.35810785e-06, 5.09779316e-03, 1.66180980e-02,
   3.46763092e-07, 3.19724760e-04, 1.02650890e-03, 6.20182713e-02,
   2.60342068e-03, 1.24946876e-01, 2.62491440e-02, 6.89327271e-02,
   1.56717104e-01, 2.76921576e-03, 7.16423163e-03, 2.05768059e-02]
    
x=module.set_vars(*x)
module.time(x),module.memory(x)

(0.09470976999335906, 0.0708826094768819)